In [1]:
import tensorflow as tf
from PIL import Image
tf.__version__

'2.0.0-rc1'

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = (x_train-127.5)/127.5
x_test = (x_test-127.5)/127.5

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

In [4]:
train_ds = train_ds.shuffle(10000).batch(32)
# suffle表示隨機選取,並分為32batch
train_ds

<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.float64, tf.uint8)>

In [5]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.shuffle(10000).batch(32)

In [6]:
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Model

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')    
    self.d2 = Dense(10, activation='softmax')

  def call(self, x): 
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
#@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [9]:
#@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [10]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1, Loss: 0.31201353669166565, Accuracy: 90.61833953857422, Test Loss: 0.16817571222782135, Test Accuracy: 94.87000274658203
Epoch 2, Loss: 0.15960127115249634, Accuracy: 95.09833526611328, Test Loss: 0.12245915830135345, Test Accuracy: 96.22000122070312
Epoch 3, Loss: 0.12459107488393784, Accuracy: 96.20000457763672, Test Loss: 0.12892121076583862, Test Accuracy: 96.0
Epoch 4, Loss: 0.10678993165493011, A